![Solar panels](https://stat.mozo.com.au/redactor/pictures/9794/how-does-solar-energy-work_original.jpg)


# This Notebook will cover -
1. Import data
2. Understanding Solar Power generation data
3. Scale data to 34 days
4. Relation between WEATHER (INPUT) and POWER (OUTPUT)?


<font size="+2" color=chocolate ><b>Please Upvote my kernel if you like my work.</b></font>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:

        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import Data

In [ ]:
#Import Data
power1 = pd.read_csv("/kaggle/input/solar-power-generation-data/Plant_1_Generation_Data.csv")
weather1 = pd.read_csv("/kaggle/input/solar-power-generation-data/Plant_1_Weather_Sensor_Data.csv")

power2 = pd.read_csv("/kaggle/input/solar-power-generation-data/Plant_2_Generation_Data.csv")
weather2 = pd.read_csv("/kaggle/input/solar-power-generation-data/Plant_2_Weather_Sensor_Data.csv")


## Understand Solar Power Generation Data

**Assumptions** (without any preprocessing)
1. This data is a **34 days** reading of a solar plant.
2. That means, the plant can only make electricity at daylight and not in nights. (should be up and down curves)
3. And this should be reflected in the **power2** Dataframe variables.


In [ ]:
# Look at data 
power2.head()

In [ ]:
weather2.head()

In [ ]:
# Notice the total entries (Not 34)
power2.info()
weather2.info()

In [ ]:
# Let's visualize it anyway.
fig, ax = plt.subplots(2,1, figsize=(20,8))
ax[0].set_title("POWER PLANT 2 DAILY CAPACITY")  
sns.lineplot(data=[power2["DC_POWER"],power2["AC_POWER"]], ax=ax[0], palette="tab20", linewidth=1)
ax[1].set_title("WEATHER ON POWER PLANT 2")  
sns.lineplot(data=[weather2["AMBIENT_TEMPERATURE"],weather2["MODULE_TEMPERATURE"],weather2["IRRADIATION"]], ax=ax[1], palette="tab20", linewidth=2.5)

**Conclusion** (For Power2) (*gathered from inverters*) 
1. Every spike represents powerplant generating power.
2. Notice: There are **34 spikes in the graph (one for everyday).**
3. **`AC_POWER` & `DC_POWER` are highly correlated.**

**Conclusion** (For Weather2) *(gathered from sensors)*
1. Every spike represents daylight.
2. Notice: There are **34 spikes in the graph (one for everyday).**
3. **`AMBIENT_TEMPERATURE` & `MODULE_TEMPRATURE` are highly correlated.**
4. `IRRADITAION` definitely seems to be giving small flucations on every spike. (might get more info. when scaling)

**Conclusion** (Both Power2 & Weather2)
1. Data gives power plant's **performance on daily basis.** 
2. `power2` have 67698 entries & `weather2` has 3259 entries for 34 days **(should be converted to 34 days)**. 
3. Even though **Output -> `power2` has 20 times more data than Input ->`weather2`** they are oscillating in similar fashion.

## Scale Data to 34 Days


In [ ]:
##### Combine the data on the basis of dates only using "DATE_TIME" column #####

## 1. Create a DATE (date format) column using "DATE_TIME"(object format) column 
df_date= weather2["DATE_TIME"].str.split()
x=[]
for d in df_date:
    val= d[0]
    x.append(val)
weather2["DATE"]= pd.DataFrame(x)
weather2.drop(["DATE_TIME"], axis=1, inplace=True)


df_date= power2["DATE_TIME"].str.split()
x=[]
for d in df_date:
    val= d[0]
    x.append(val)
power2["DATE"]= pd.DataFrame(x)
power2.drop(["DATE_TIME"], axis=1, inplace=True)




In [ ]:
## 2. Group by date and MEan the rest
groupMeanWeather2 = weather2.groupby("DATE").max().reset_index()
groupMeanPower2 = power2.groupby("DATE").max().reset_index()

# Look at the 34 Entries for both the tables
groupMeanWeather2.info()
groupMeanPower2.info()

In [ ]:
def getNormalized(num):
    return num/(num.max()+num.min())

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20,4))
w_amb = getNormalized(groupMeanWeather2["AMBIENT_TEMPERATURE"])
mod_temp = getNormalized(groupMeanWeather2["MODULE_TEMPERATURE"])
w_irr = getNormalized(groupMeanWeather2["IRRADIATION"]) 

sns.lineplot(data=[w_amb,mod_temp,w_irr], ax=ax, palette="tab20", linewidth=1.5,dashes=False).set_title("Scaled Weather parameters)")
plt.show()

**Conclusion** Now we can easily see  clear relation in weather data between these 3 variables over the period of 34 days.

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20,4))
p_dc = getNormalized(groupMeanPower2["DC_POWER"])
p_ac = getNormalized(groupMeanPower2["AC_POWER"])
p_dy = getNormalized(groupMeanPower2["DAILY_YIELD"])
p_ty = getNormalized(groupMeanPower2["TOTAL_YIELD"]) 

sns.lineplot(data=[p_ac,p_dc-0.01,p_dy,p_ty], ax=ax, palette="tab20", linewidth=1.5,dashes=False).set_title("Scaled Power Parameters")
plt.show()

**Conclusion** Also, there is a clear relation in power data between AC_POWER, DC_POWER and DAILY_YIELD over the period of 34 days.

## **Relation between WEATHER (*INPUT*) and POWER (*OUTPUT*)?** 

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20,8))

sns.lineplot(data=[p_ac,w_amb,mod_temp,w_irr], ax=ax, palette="tab20", linewidth=1,dashes=False).set_title("Scaled Weather Vs Power parameters")
plt.show()



In [ ]:

sns.jointplot(x=p_ac,y=w_irr ,kind="reg")
sns.jointplot(x=w_amb,y=mod_temp, kind="kde")

In [ ]:
newdata =  pd.DataFrame({"AC": p_ac.values,
                        "AMB": w_amb.values,
                        "MOD":mod_temp.values,
                        "IRR":w_irr.values})


In [ ]:
newdata.describe()

In [ ]:
sns.pairplot(data =newdata)

**************************************************************************************************

In [ ]:
# drawthis = newdata.pivot("DATE", "AMBIENT_TEMPERATURE", "DC_POWER")

# Draw a heatmap with the numeric values in each cell
f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(newdata, annot=True, fmt="d", linewidths=.5, ax=ax)

In [ ]:
# Group by date and Sum the rest
groupweather2 = weather2.groupby("DATE").sum().reset_index()
grouppower2 = power2.groupby("DATE").sum().reset_index()

In [ ]:
# Group beydate and mean the reset 
groupweather2mean = weather2.groupby("DATE").mean().reset_index()
grouppower2mean = power2.groupby("DATE").mean().reset_index()

In [ ]:
groupweather2mean.info()
grouppower2mean.info()

In [ ]:
groupweather2.info()
grouppower2.info()

In [ ]:
# Merged onDate
merged2 = pd.merge(groupweather2,grouppower2,on="DATE")
merged2.info()

# Mean Merged Date
meanmerged2 = pd.merge(groupweather2mean,grouppower2mean,on="DATE")
meanmerged2.info()

In [ ]:
meanmerged2.head()

In [ ]:
meanmerged2['DATE'] = pd.to_datetime(meanmerged2['DATE'], format='%Y-%m-%d') 
meanmerged2.info()

In [ ]:

fig, ax = plt.subplots(1,1, figsize=(20,4))
p_dc = (meanmerged2["DC_POWER"]/280)-0.4
p_ac = (meanmerged2["AC_POWER"]/280)-0.4

p_dy = meanmerged2["DAILY_YIELD"]/10000
w_amb = ((meanmerged2["AMBIENT_TEMPERATURE"]/35 )-0.6)*3
mod_temp = ((meanmerged2["MODULE_TEMPERATURE"]/60 ) - 0.3)*3
w_irr = meanmerged2["IRRADIATION"]*3
sns.lineplot(data=[p_ac,p_dc,w_amb,mod_temp,w_irr], ax=ax, palette="tab20", linewidth=2.5)
plt.show()

In [ ]:
def getNormalized(num):
    return num/(num.max()+num.min())
    

In [ ]:

fig, ax = plt.subplots(1,1, figsize=(20,4))
p_dc = getNormalized(meanmerged2["DC_POWER"])
p_ac = getNormalized(meanmerged2["AC_POWER"])
# p_dy = meanmerged2["DAILY_YIELD"]/10000
w_amb = getNormalized(meanmerged2["AMBIENT_TEMPERATURE"])
mod_temp = getNormalized(meanmerged2["MODULE_TEMPERATURE"])
w_irr = getNormalized(meanmerged2["IRRADIATION"]) 
p_dy = getNormalized(meanmerged2["DAILY_YIELD"])
p_ty = getNormalized(meanmerged2["TOTAL_YIELD"]) 

sns.lineplot(data=[p_ac,p_dc,w_amb,mod_temp,w_irr,p_dy,p_ty], ax=ax, palette="tab20", linewidth=1.5,dashes=False)
plt.show()

In [ ]:
meanmerged2.describe()

In [ ]:

p_dy = getNormalized(meanmerged2["DAILY_YIELD"])
p_ty =  getNormalized(meanmerged2["TOTAL_YIELD"]) 

# w_amb = ((meanmerged2["AMBIENT_TEMPERATURE"]/35 )-0.6)*3
# mod_temp = ((meanmerged2["MODULE_TEMPERATURE"]/60 ) - 0.3)*3
# w_irr = meanmerged2["IRRADIATION"]*3
sns.lineplot(data=[p_dy,p_ty], ax=ax, palette="tab20", linewidth=2.5)
plt.show()

In [ ]:
meanScaledData = pd.DataFrame({"DC_POWER":p_dc,
                                   "AC_POWER":p_ac,
                                   "AMBIENT_TEMPERATURE":w_amb,
                                   "MODULE_TEMPERATURE":mod_temp,
                                   "IRRADIATION":w_irr})
meanScaledData.info()
meanScaledData.describe()

In [ ]:
sns.pairplot(data = meanScaledData)

In [ ]:
fig_dims = (8, 8)
fig, ax = plt.subplots(figsize=fig_dims)

corr = meanScaledData.corr()
sns.heatmap(corr,ax = ax, annot=True)

plt.show()

In [ ]:
# fig, ax = plt.subplots(1,2, figsize=(8,8))
sns.jointplot(x="IRRADIATION",y="DC_POWER",data=meanScaledData ,kind="reg")
sns.jointplot(x="AMBIENT_TEMPERATURE",y="MODULE_TEMPERATURE",data=meanScaledData, kind="kde")
# plt.show()

<font size="+2" color=chocolate ><b>Please Upvote my kernel if you like my work.</b></font>